In [1]:
import pandas as pd
import numpy as np
from typing import (List, Tuple, Dict, Union)
from data_service import (get_shibor_data, get_interpld_shibor,
                          query_china_shibor_all)
from collections import namedtuple
from scr.calc_func import (_get_near_or_next_options, _get_free_rate,
                           get_daily_vix)


本篇算法来源：
>《20180707_东北证券_金融工程_市场波动风险度量：vix与skew指数构建与应用》

In [2]:
opt_data = pd.read_csv('opt_data.csv', index_col=[0])
# df_rate = pd.read_csv('shibor_df.csv',index_col=0) # from jqdata

shibor_df = get_shibor_data('2015-02-09', '2022-05-27')
rate_df = get_interpld_shibor(shibor_df)

In [10]:
# 获取每日的近月和次近月期权合约
filter_opt_data: pd.DataFrame = opt_data.groupby(
    'date', group_keys=False).apply(_get_near_or_next_options)
filter_opt_data['date'] = pd.to_datetime(filter_opt_data['date'])

# 获取每日
maturity_ser: pd.Series = filter_opt_data.groupby('date').apply(
    lambda x: x['maturity'].unique())

# 近月、次近月
sel_rate: pd.Series = rate_df.loc[maturity_ser.index].apply(
    lambda x: _get_free_rate(x, np.min(maturity_ser.loc[x.name]),
                             np.max(maturity_ser.loc[x.name])),
    axis=1)

# 根据maturity对齐shibor
maturity_align, shibor_algin = maturity_ser.align(sel_rate,
                                                  axis=0,
                                                  join='left')

# 储存中间变量
df = pd.DataFrame(
    index=maturity_align.index,
    columns='near_maturity,next_maturity,near_rate,next_rate'.split(','))

df['near_maturity'] = maturity_align.apply(lambda x: np.min(x))
df['next_maturity'] = maturity_align.apply(lambda x: np.max(x))

df['near_rate'] = shibor_algin.apply(lambda x: np.min(x))
df['next_rate'] = shibor_algin.apply(lambda x: np.max(x))
df.index.names = ['date']

#opt_data['date'] = pd.to_datetime(opt_data['date'])
data_all = pd.merge(filter_opt_data, df.reset_index(), on='date', how='outer')


In [4]:
# data_all.query('date == "2015-03-11"')# ['maturity'].unique()

In [5]:
# res = get_daily_vix(data_all.query('date == "2015-03-11"'))

# near_sigma = res.sigma['near']
# next_sigma = res.sigma['next']
# near_term = res.term['near']
# next_term = res.term['next']

# calc_vix(near_sigma,next_sigma,near_term,next_term)

In [4]:
from scr.calc_func import (_build_strike_matrix, _get_min_strike_diff, calc_F,
                           _get_median_price_table)


In [11]:
ser = pd.Series(index=data_all['date'].unique(),
                data=np.empty(len(data_all['date'].unique())))
for trade, df in data_all.groupby('date'):
    ser[trade] = get_daily_vix(df)

F:2.7309
[False False False False False False False False False False False False]
contract_type     call     put    diff
exercise_price                        
2.75            0.1701  0.1892 -0.0191
2.80            0.1576  0.2266 -0.0690
2.85            0.1418  0.2523 -0.1105
2.90            0.1193  0.2870 -0.1677
2.95            0.1114  0.3249 -0.2135
3.00            0.1050  0.3796 -0.2746
3.10            0.0820  0.4576 -0.3756
3.20            0.0715  0.5440 -0.4725
3.30            0.0583  0.6215 -0.5632
3.40            0.0458  0.7127 -0.6669
3.50            0.0333  0.8078 -0.7745
3.60            0.0257  0.8887 -0.8630


ValueError: 无对应的K_0数据!

In [12]:
# 获取对应的期权信息
## 近月
near_df: pd.DataFrame = df[df['maturity'] == df['near_maturity']]
near_strike_matrix: pd.DataFrame = _build_strike_matrix(near_df)

near_strike_ser: pd.Series = _get_min_strike_diff(near_strike_matrix)

## 近月无风险收益等
near_term: float = near_df['near_maturity'].iloc[0]
near_term_rate: float = near_df['near_rate'].iloc[0]

## 次近月
next_df: pd.DataFrame = df[df['maturity'] == df['next_maturity']]
next_strike_matrix: pd.DataFrame = _build_strike_matrix(
    df[df['maturity'] == df['next_maturity']])

next_strike_ser: pd.Series = _get_min_strike_diff(next_strike_matrix)
## 次近月无风险收益等
next_term: float = next_df['next_maturity'].iloc[0]
next_term_rate: float = next_df['next_rate'].iloc[0]

# 计算远期价格
near_F: float = calc_F(near_strike_ser['exercise_price'], near_term_rate,
                       near_term, near_strike_ser['call'],
                       near_strike_ser['put'])

next_F: float = calc_F(next_strike_ser['exercise_price'], next_term_rate,
                       next_term, next_strike_ser['call'],
                       next_strike_ser['put'])

# 计算中间价格表
near_median_table, near_K_0 = _get_median_price_table(near_strike_matrix,
                                                      near_F)


F:2.7309
[False False False False False False False False False False False False]
contract_type     call     put    diff
exercise_price                        
2.75            0.1701  0.1892 -0.0191
2.80            0.1576  0.2266 -0.0690
2.85            0.1418  0.2523 -0.1105
2.90            0.1193  0.2870 -0.1677
2.95            0.1114  0.3249 -0.2135
3.00            0.1050  0.3796 -0.2746
3.10            0.0820  0.4576 -0.3756
3.20            0.0715  0.5440 -0.4725
3.30            0.0583  0.6215 -0.5632
3.40            0.0458  0.7127 -0.6669
3.50            0.0333  0.8078 -0.7745
3.60            0.0257  0.8887 -0.8630


ValueError: 无对应的K_0数据!

In [19]:
near_strike_matrix.reset_index().apply(lambda x:calc_F(x['exercise_price'],x['']))

contract_type,exercise_price,call,put,diff
0,2.75,0.1701,0.1892,-0.0191
1,2.80,0.1576,0.2266,-0.0690
2,2.85,0.1418,0.2523,-0.1105
3,2.90,0.1193,0.2870,-0.1677
4,2.95,0.1114,0.3249,-0.2135
5,3.00,0.1050,0.3796,-0.2746
6,3.10,0.0820,0.4576,-0.3756
7,3.20,0.0715,0.5440,-0.4725
8,3.30,0.0583,0.6215,-0.5632
9,3.40,0.0458,0.7127,-0.6669


In [15]:
next_strike_matrix,next_strike_matrix

(contract_type     call     put    diff
 exercise_price                        
 2.80            0.2220  0.2667 -0.0447
 2.85            0.1909  0.3016 -0.1107
 2.90            0.1738  0.3296 -0.1558
 2.95            0.1596  0.3691 -0.2095
 3.00            0.1518  0.3980 -0.2462
 3.10            0.1272  0.4796 -0.3524
 3.20            0.1117  0.5750 -0.4633,
 contract_type     call     put    diff
 exercise_price                        
 2.80            0.2220  0.2667 -0.0447
 2.85            0.1909  0.3016 -0.1107
 2.90            0.1738  0.3296 -0.1558
 2.95            0.1596  0.3691 -0.2095
 3.00            0.1518  0.3980 -0.2462
 3.10            0.1272  0.4796 -0.3524
 3.20            0.1117  0.5750 -0.4633)

In [17]:
# watch_date = '2015-02-09'
# COND = (opt_data['date'] == watch_date)
# filter_df = opt_data[COND]

# # 模拟单日
# tmp_df = filter_df.groupby('date',
#                            group_keys=False).apply(_get_near_or_next_options)

# # 获取近月、次近月
# near_term, next_term = np.sort(tmp_df['maturity'].unique())

# # 获取当日对应的无风险收益
# near_term_rate, next_term_rate = _get_free_rate(rate_df.loc[watch_date],
#                                                 near_term, next_term)

# # 获取对应的期权信息
# ## 近月
# near_strike_matrix: pd.DataFrame = _build_strike_matrix(
#     tmp_df[tmp_df['maturity'] == near_term])

# near_strike_ser = _get_min_strike_diff(near_strike_matrix)

# ## 次近月
# next_strike_matrix: pd.DataFrame = _build_strike_matrix(
#     tmp_df[tmp_df['maturity'] == next_term])

# next_strike_ser = _get_min_strike_diff(next_strike_matrix)

# # 计算远期价格
# near_F = calc_F(near_strike_ser['exercise_price'], near_term_rate, near_term,
#                 near_strike_ser['call'], near_strike_ser['put'])

# next_F = calc_F(next_strike_ser['exercise_price'], next_term_rate, next_term,
#                 next_strike_ser['call'], next_strike_ser['put'])

# # 计算中间价格表
# near_median_table, near_K_0 = _get_median_price_table(near_strike_matrix,
#                                                       near_F)
# next_median_table, next_K_0 = _get_median_price_table(next_strike_matrix,
#                                                       next_F)

# # 计算delta_k
# near_delta_k = calc_delta_k_table(near_median_table)
# next_delta_k = calc_delta_k_table(next_median_table)

# # 计算simma
# near_sigma = calc_sigma(near_K_0, near_median_table.index._values,
#                         near_delta_k.values, near_median_table.values, near_F,
#                         near_term_rate, near_term)

# next_sigma = calc_sigma(next_K_0, next_median_table.index._values,
#                         next_delta_k.values, next_median_table.values, next_F,
#                         next_term_rate, next_term)
# # 计算vix
# vix = calc_vix(near_sigma, next_sigma, near_term, next_term)

# VIX计算中间参数变量表
# tmp_variable = pd.concat((near_median_table,near_delta_k),axis=1).reset_index()
# tmp_variable.columns = ['K','Q(K)','Delta_K']
# tmp_variable['F'] = F_1
# tmp_variable['T'] = near_term
# tmp_variable['R'] = near_term_rate


In [ ]:
# near_term_opt 近期
# next_term_opt 下期